In [1]:
import numpy as np
import pandas as pd
import os
import collections as c
import matplotlib.pyplot as plt
os.getcwd()

'/home/abhishek/Documents/visualization/myapp'

In [2]:
base = '/home/abhishek/Documents/'

In [3]:
## save the sure_id dataframe into csv file.

os.chdir('/home/abhishek/Documents/visualization/myapp/id')
sure_id = pd.read_csv('sure_id.csv')
sure_id = sure_id[['co_code', 'industry_type','incorporation_year', 'nse']]
sure_id

,co_code,industry_type,incorporation_year,nse
0,113,Business services & consultancy,2000.0,AAATECH
1,363,Business services & consultancy,1983.0,ABGHEAVY
2,3052,Business services & consultancy,1983.0,ATNINTER
3,4030,Business services & consultancy,1987.0,AARVI
4,8628,Business services & consultancy,1995.0,AIRAN
...,...,...,...,...
2420,98646,Online marketplaces,1999.0,INDIAMART
2421,412105,Online marketplaces,2010.0,ZOMATO
2422,136444,Diversified automobile,1945.0,M&M
2423,444974,Road transport infrastructure services,2002.0,MEP


In [4]:
os.chdir(base+'/visualization/assignment2/fs')

names = [
    'co_code' ,'year', 'sales',  'retained_profits',
    'total_assets', 'gross_fixed_assets', 
    'investment_outside_india',
    'export_goods', 'export_serv'
    
        ]

data = pd.read_csv(
    'fs_dat.txt', sep='|', header=0,
    names=names,  usecols=[0,2,5,13,14,15,17,22,23]
)


## convert all NaN values in investment_outside_india and rnd_expenses
## and exports_serv and exports_goods to zero.
data.investment_outside_india = data.investment_outside_india.fillna(0)
data.export_goods = data.export_goods.fillna(0)
data.export_serv = data.export_serv.fillna(0)


## remove all rows with any NaN values
data.dropna(axis=0, how='any', inplace=True)

## convert all values from string to float
data.year = data.year.astype(float)

## date in given in yyyymmdd format, so i converted it into yyyy format
data.year=data.year.floordiv(10000)

## create the exports column and remove the export_goods and export_serv colns
data = data.assign(exports = data.export_goods+data.export_serv)
data.drop(columns = ['export_goods', 'export_serv'], inplace = True)

## 5 firms out of a total of 1,58,200 firms have released more than one annual financial statement in an year
## so if firms have n financial statements in a year, I removed n-1 of them
data.drop_duplicates(subset=['co_code','year'], keep='first',inplace=True)

## only consider data up until 2012 with sales&total_assets more than 1 crore
data = data[(data.year>2000)&(data.year<2023)&( (data.sales>100) | (data.total_assets>100) )]
data

,co_code,year,sales,retained_profits,total_assets,gross_fixed_assets,investment_outside_india,exports
4,11,2001.0,442.2,19.9,779.4,405.6,0.0,56.2
5,11,2002.0,401.5,-0.6,829.9,422.7,0.0,71.0
6,11,2003.0,462.5,-8.9,850.5,479.6,0.0,78.1
7,11,2004.0,533.6,-60.8,768.5,592.5,0.0,84.2
8,11,2005.0,578.3,-63.2,698.5,666.3,0.0,93.7
...,...,...,...,...,...,...,...,...
163199,663148,2020.0,305.4,0.6,184.4,0.6,0.0,0.0
163200,663148,2021.0,310.1,0.2,248.0,0.7,0.0,0.0
163204,663918,2020.0,115.7,-0.1,58.9,37.8,0.0,0.0
163205,663918,2021.0,131.2,1.0,73.7,42.5,0.0,0.0


In [5]:
ds = pd.merge(left = sure_id, right = data, how = 'inner', on = 'co_code')

ds

,co_code,industry_type,incorporation_year,nse,year,sales,retained_profits,total_assets,gross_fixed_assets,investment_outside_india,exports
0,113,Business services & consultancy,2000.0,AAATECH,2014.0,132.3,1.2,47.7,9.2,0.0,0.0
1,113,Business services & consultancy,2000.0,AAATECH,2015.0,134.9,1.0,45.3,9.2,0.0,0.0
2,113,Business services & consultancy,2000.0,AAATECH,2016.0,127.2,1.0,49.0,9.8,0.0,0.0
3,113,Business services & consultancy,2000.0,AAATECH,2017.0,123.2,0.9,52.4,9.8,0.0,0.0
4,113,Business services & consultancy,2000.0,AAATECH,2018.0,102.3,10.0,104.7,12.3,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
34239,570802,Irrigation,2004.0,RMDRIP,2017.0,283.4,13.6,238.1,100.6,0.0,0.0
34240,570802,Irrigation,2004.0,RMDRIP,2018.0,426.3,15.7,325.0,115.9,0.0,0.0
34241,570802,Irrigation,2004.0,RMDRIP,2019.0,222.9,-54.4,269.6,122.0,0.0,0.0
34242,570802,Irrigation,2004.0,RMDRIP,2020.0,232.4,28.0,273.1,125.7,0.0,0.0


In [6]:
os.chdir('/home/abhishek/Documents/visualization/myapp')

In [7]:
ds = pd.merge(left = sure_id, right = data, how = 'inner', on = 'co_code')

name = [ 'gross_fixed_assets', 'investment_outside_india', 'exports']


ds.drop(columns=name,inplace=True)




ds.to_csv('dataset.csv', index = False)

In [10]:
ds

,co_code,industry_type,incorporation_year,nse,year,sales,retained_profits,total_assets
0,113,Business services & consultancy,2000.0,AAATECH,2014.0,132.3,1.2,47.7
1,113,Business services & consultancy,2000.0,AAATECH,2015.0,134.9,1.0,45.3
2,113,Business services & consultancy,2000.0,AAATECH,2016.0,127.2,1.0,49.0
3,113,Business services & consultancy,2000.0,AAATECH,2017.0,123.2,0.9,52.4
4,113,Business services & consultancy,2000.0,AAATECH,2018.0,102.3,10.0,104.7
...,...,...,...,...,...,...,...,...
34239,570802,Irrigation,2004.0,RMDRIP,2017.0,283.4,13.6,238.1
34240,570802,Irrigation,2004.0,RMDRIP,2018.0,426.3,15.7,325.0
34241,570802,Irrigation,2004.0,RMDRIP,2019.0,222.9,-54.4,269.6
34242,570802,Irrigation,2004.0,RMDRIP,2020.0,232.4,28.0,273.1
